37902 Foundation of Advanced Quantitative Marketing

Li Liu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
pd.options.display.max_colwidth = 1000

### Yogurt100N Sales Data

In [2]:
df=pd.read_excel("Yogurt100N.csv.xlsx")
df.describe()
df.head()

,Pan I.D.,Expend $,Income,HH Size,IPT,Quantity,Brand 1,Brand 2,Brand 3,Brand 4,Feature 1,Feature 2,Feature 3,Feature 4,Price 1,Price 2,Price 3,Price 4,PanelistFirstObs
0,1,40.900002,9,2,5,2,0,0,0,1,0,0,0,0,0.108,0.081,0.061,0.079,1
1,1,16.809999,9,2,5,2,0,1,0,0,0,0,0,0,0.108,0.098,0.064,0.075,0
2,1,4.060000,9,2,1,2,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086,0
3,1,34.459999,9,2,4,2,0,1,0,0,0,0,0,0,0.108,0.098,0.061,0.086,0
4,1,8.390000,9,2,7,2,0,1,0,0,0,0,0,0,0.125,0.098,0.049,0.079,0


### Simple Logit on Yogurt Data

In [3]:
def crit(params):

    a1,a2,a3,bf,bp=params
    ev1=np.exp(a1+bf*df['Feature 1']+bp*df['Price 1'])
    ev2=np.exp(a2+bf*df['Feature 2']+bp*df['Price 2'])
    ev3=np.exp(a3+bf*df['Feature 3']+bp*df['Price 3'])
    ev4=np.exp(0+bf*df['Feature 4']+bp*df['Price 4'])
    denom=ev1+ev2+ev3+ev4
    global p1,p2,p3,p4
    p1,p2,p3,p4=ev1/denom,ev2/denom,ev3/denom,ev4/denom
    pc=(ev1*df['Brand 1']+ev2*df['Brand  2']+ev3*df['Brand 3']+ev4*df['Brand 4'])/denom
    Inpc=np.log(pc)
    LL=np.sum(Inpc)
    return -LL
a1,a2,a3,bf,bp=1,1,1,1,1 #Initalization
params_init = np.array([a1,a2,a3,bf,bp])
results = opt.minimize(crit, params_init)
a1,a2,a3,bf,bp = results.x
print(" a1 (Intrinsic brand preference for Brand 1):",a1,"\n",
      "a2 (Intrinsic brand preference for Brand 2):",a2,"\n",
      "a3 (Intrinsic brand preference for Brand 3):",a3,"\n",
      "bf (Coefficients for feature variable):",bf,"\n",
      "bp (Coefficients for price variable):",bp,"\n",
      "Maximized Log Likelihood:",-results.fun)

 a1 (Intrinsic brand preference for Brand 1): 1.3877493848693059 
 a2 (Intrinsic brand preference for Brand 2): 0.6435046305879636 
 a3 (Intrinsic brand preference for Brand 3): -3.0861119572117355 
 bf (Coefficients for feature variable): 0.4874149107851659 
 bp (Coefficients for price variable): -37.057782766093105 
 Maximized Log Likelihood: -2658.5566975071233


In [4]:
#Own elasticities
e11=np.mean((bf*df['Feature 1']+bp*df['Price 1'])*(1-p1))
e22=np.mean((bf*df['Feature 2']+bp*df['Price 2'])*(1-p2))
e33=np.mean((bf*df['Feature 3']+bp*df['Price 3'])*(1-p3))
e44=np.mean((bf*df['Feature 4']+bp*df['Price 4'])*(1-p4))

In [5]:
#Property of Logit Model: for all j != k, the cross elasticity will be the same.
e21=e31=e41=np.mean(-(bf*df['Feature 1']+bp*df['Price 1'])*p1)
e12=e32=e42=np.mean(-(bf*df['Feature 2']+bp*df['Price 2'])*p2)
e13=e23=e43=np.mean(-(bf*df['Feature 3']+bp*df['Price 3'])*p3)
e14=e24=e34=np.mean(-(bf*df['Feature 4']+bp*df['Price 4'])*p4)

In [6]:
mat=pd.DataFrame({"Brand 1":[e11,e21,e31,e41], "Brand2":[e12,e22,e32,e42],
                  "Brand 3":[e13,e23,e33,e43],'Brand 4':[e14,e24,e34,e44]})
mat.index=["Brand 1","Brand 2", "Brand 3", "Brand 4"]
print("Elasiticy Matrix with Simple Logit Model")
mat

Elasiticy Matrix with Simple Logit Model


,Brand 1,Brand2,Brand 3,Brand 4
Brand 1,-2.688666,1.164621,0.052607,0.651862
Brand 2,1.221561,-1.837507,0.052607,0.651862
Brand 3,1.221561,1.164621,-1.916245,0.651862
Brand 4,1.221561,1.164621,0.052607,-2.276253


### Nested Logit Model on Yogurt Data

#### 1. brands 1~3 in one nest and 4 in another 

In [7]:
def nestedlogit(params):

    a1,a2,a3,bf,bp,theta=params 
    rho=np.exp(theta)/(np.exp(theta)+1)
    ev1=np.exp((a1+bf*df['Feature 1']+bp*df['Price 1'])/rho)
    ev2=np.exp((a2+bf*df['Feature 2']+bp*df['Price 2'])/rho)
    ev3=np.exp((a3+bf*df['Feature 3']+bp*df['Price 3'])/rho)
    ev4=np.exp(0+bf*df['Feature 4']+bp*df['Price 4'])
    denom=ev1+ev2+ev3
    global P1,P2,P3,P4

    P4=ev4/(np.power(denom,rho)+ev4)
    P1=(1-P4)*ev1/denom
    P2=(1-P4)*ev2/denom
    P3=(1-P4)*ev3/denom
        
    pc=(P1*df['Brand 1']+P2*df['Brand  2']+P3*df['Brand 3']+P4*df['Brand 4'])
    Inpc=np.log(pc)
    LL=np.sum(Inpc)
    return -LL

In [8]:
a1,a2,a3,bf,bp,theta=1,1,1,1,1,1 #Initalization
params_init = np.array([a1,a2,a3,bf,bp,theta])
results = opt.minimize(nestedlogit, params_init)
a1,a2,a3,bf,bp,theta = results.x
print(" a1 (Intrinsic brand preference for Brand 1):",a1,"\n",
      "a2 (Intrinsic brand preference for Brand 2):",a2,"\n",
      "a3 (Intrinsic brand preference for Brand 3):",a3,"\n",
      "bf (Coefficients for feature variable):",bf,"\n",
      "bp (Coefficients for price variable):",bp,"\n",
      "rho (Correlation variable):",np.exp(theta)/(np.exp(theta)+1),"\n",
      "Maximized Log Likelihood:",-results.fun)

 a1 (Intrinsic brand preference for Brand 1): 1.3816689432731886 
 a2 (Intrinsic brand preference for Brand 2): 0.8394218694834024 
 a3 (Intrinsic brand preference for Brand 3): -1.6585023784518325 
 bf (Coefficients for feature variable): 0.3744679884335011 
 bp (Coefficients for price variable): -26.58112038547648 
 rho (Correlation variable): 0.6433848076286417 
 Maximized Log Likelihood: -2653.7645999847773


In [9]:
vcv_mle = results.hess_inv
stderr_a1_mle = np.sqrt(vcv_mle[0,0])
stderr_a2_mle = np.sqrt(vcv_mle[1,1])
stderr_a3_mle = np.sqrt(vcv_mle[2,2])
stderr_bf_mle = np.sqrt(vcv_mle[3,3])
stderr_bp_mle = np.sqrt(vcv_mle[4,4])
stderr_theta_mle = np.sqrt(vcv_mle[5,5])

print('Standard error for a1 estimate = ', stderr_a1_mle)
print('Standard error for a2 estimate = ', stderr_a2_mle)
print('Standard error for a3 estimate = ', stderr_a3_mle)
print('Standard error for bf estimate = ', stderr_bf_mle)
print('Standard error for bp estimate = ', stderr_bp_mle)
print('Standard error for theta estimate = ', stderr_theta_mle)

Standard error for a1 estimate =  0.07125745085397713
Standard error for a2 estimate =  0.0498364392238186
Standard error for a3 estimate =  0.023037025008121905
Standard error for bf estimate =  0.08637968526841336
Standard error for bp estimate =  1.4536600440077279
Standard error for theta estimate =  0.1084251269080594


#### 2. brands 1 and 4 in one nest; 2 and 3 in another

In [10]:
def nestedlogit2(params):

    a1,a2,a3,bf,bp,theta1,theta2=params
    rho1=np.exp(theta1)/(np.exp(theta1)+1)
    rho2=np.exp(theta2)/(np.exp(theta2)+1)
    
    #Nest1 (Brand 1 and 4)
    ev1=np.exp((a1+bf*df['Feature 1']+bp*df['Price 1'])/rho1)
    ev4=np.exp((0+bf*df['Feature 4']+bp*df['Price 4'])/rho1)
    denom1=ev1+ev4

    #Nest2 (Brand 2 and 3)
    ev2=np.exp((a2+bf*df['Feature 2']+bp*df['Price 2'])/rho2)
    ev3=np.exp((a3+bf*df['Feature 3']+bp*df['Price 3'])/rho2)
    denom2=ev2+ev3
    
    Nest1=np.power(denom1,rho1)/(np.power(denom1,rho1)+np.power(denom2,rho2))
    Nest2=np.power(denom2,rho2)/(np.power(denom1,rho1)+np.power(denom2,rho2))

    P1=Nest1*ev1/denom1
    P2=Nest2*ev2/denom2
    P3=Nest2*ev3/denom2
    P4=Nest1*ev4/denom1
     
    pc=(P1*df['Brand 1']+P2*df['Brand  2']+P3*df['Brand 3']+P4*df['Brand 4'])
    Inpc=np.log(pc)
    LL=np.sum(Inpc)
    return -LL

In [11]:
a1,a2,a3,bf,bp,theta1,theta2=1,1,1,1,1,1,1 #Initalization
params_init = np.array([a1,a2,a3,bf,bp,theta1,theta2])
results = opt.minimize(nestedlogit2, params_init)
a1,a2,a3,bf,bp,theta1,theta2 = results.x
print(" a1 (Intrinsic brand preference for Brand 1):",a1,"\n",
      "a2 (Intrinsic brand preference for Brand 2):",a2,"\n",
      "a3 (Intrinsic brand preference for Brand 3):",a3,"\n",
      "bf (Coefficients for feature variable):",bf,"\n",
      "bp (Coefficients for price variable):",bp,"\n",
      "rho (Correlation variable 1):",np.exp(theta1)/(np.exp(theta1)+1),"\n",
      "rho (Correlation variable 2):",np.exp(theta2)/(np.exp(theta2)+1),"\n",
      "Maximized Log Likelihood:",-results.fun)

 a1 (Intrinsic brand preference for Brand 1): 1.2704362491822534 
 a2 (Intrinsic brand preference for Brand 2): 0.5967295980737131 
 a3 (Intrinsic brand preference for Brand 3): -1.955107545333905 
 bf (Coefficients for feature variable): 0.44043450580060095 
 bp (Coefficients for price variable): -33.755240518991044 
 rho (Correlation variable 1): 0.9169151375527343 
 rho (Correlation variable 2): 0.5607146318939851 
 Maximized Log Likelihood: -2652.8512628377266


In [12]:
vcv_mle = results.hess_inv
stderr_a1_mle = np.sqrt(vcv_mle[0,0])
stderr_a2_mle = np.sqrt(vcv_mle[1,1])
stderr_a3_mle = np.sqrt(vcv_mle[2,2])
stderr_bf_mle = np.sqrt(vcv_mle[3,3])
stderr_bp_mle = np.sqrt(vcv_mle[4,4])
stderr_theta_mle = np.sqrt(vcv_mle[5,5])

print('Standard error for a1 estimate = ', stderr_a1_mle)
print('Standard error for a2 estimate = ', stderr_a2_mle)
print('Standard error for a3 estimate = ', stderr_a3_mle)
print('Standard error for bf estimate = ', stderr_bf_mle)
print('Standard error for bp estimate = ', stderr_bp_mle)
print('Standard error for theta estimate = ', stderr_theta_mle)

Standard error for a1 estimate =  0.08758688439542385
Standard error for a2 estimate =  0.061878752812136985
Standard error for a3 estimate =  0.16247679412323798
Standard error for bf estimate =  0.11976577605634205
Standard error for bp estimate =  1.5165678362983608
Standard error for theta estimate =  0.6269595968830872


#### 3. brands 1 and 2 in one nest; 3 and 4 in another

In [13]:
def nestedlogit3(params):

    a1,a2,a3,bf,bp,theta1,theta2=params
    rho1=np.exp(theta1)/(np.exp(theta1)+1)
    rho2=np.exp(theta2)/(np.exp(theta2)+1)
    
    #Nest1 (Brand 1 and 2)
    ev1=np.exp((a1+bf*df['Feature 1']+bp*df['Price 1'])/rho1)
    ev2=np.exp((a2+bf*df['Feature 2']+bp*df['Price 2'])/rho1)
    denom1=ev1+ev2

    #Nest2 (Brand 3 and 4)
    ev3=np.exp((a3+bf*df['Feature 3']+bp*df['Price 3'])/rho2)
    ev4=np.exp((0+bf*df['Feature 4']+bp*df['Price 4'])/rho2)
    denom2=ev3+ev4
      
    Nest1=np.power(denom1,rho1)/(np.power(denom1,rho1)+np.power(denom2,rho2))
    Nest2=np.power(denom2,rho2)/(np.power(denom1,rho1)+np.power(denom2,rho2))

    P1=Nest1*ev1/denom1
    P2=Nest1*ev2/denom1
    P3=Nest2*ev3/denom2
    P4=Nest2*ev4/denom2
     
    pc=(P1*df['Brand 1']+P2*df['Brand  2']+P3*df['Brand 3']+P4*df['Brand 4'])
    Inpc=np.log(pc)
    LL=np.sum(Inpc)
    return -LL

In [14]:
a1,a2,a3,bf,bp,theta1,theta2=1,1,1,1,1,1,1 #Initalization
params_init = np.array([a1,a2,a3,bf,bp,theta1,theta2])
results = opt.minimize(nestedlogit3, params_init)
a1,a2,a3,bf,bp,theta1,theta2 = results.x
print(" a1 (Intrinsic brand preference for Brand 1):",a1,"\n",
      "a2 (Intrinsic brand preference for Brand 2):",a2,"\n",
      "a3 (Intrinsic brand preference for Brand 3):",a3,"\n",
      "bf (Coefficients for feature variable):",bf,"\n",
      "bp (Coefficients for price variable):",bp,"\n",
      "rho (Correlation variable 1):",np.exp(theta1)/(np.exp(theta1)+1),"\n",
      "rho (Correlation variable 2):",np.exp(theta2)/(np.exp(theta2)+1),"\n",
      "Maximized Log Likelihood:",-results.fun)

 a1 (Intrinsic brand preference for Brand 1): 1.3084060666455373 
 a2 (Intrinsic brand preference for Brand 2): 0.7343621134017146 
 a3 (Intrinsic brand preference for Brand 3): -1.929613660806728 
 bf (Coefficients for feature variable): 0.38708415089337606 
 bp (Coefficients for price variable): -28.195927795046757 
 rho (Correlation variable 1): 0.7016516377756066 
 rho (Correlation variable 2): 0.5252493668403772 
 Maximized Log Likelihood: -2654.0983981494196


In [15]:
vcv_mle = results.hess_inv
stderr_a1_mle = np.sqrt(vcv_mle[0,0])
stderr_a2_mle = np.sqrt(vcv_mle[1,1])
stderr_a3_mle = np.sqrt(vcv_mle[2,2])
stderr_bf_mle = np.sqrt(vcv_mle[3,3])
stderr_bp_mle = np.sqrt(vcv_mle[4,4])
stderr_theta_mle = np.sqrt(vcv_mle[5,5])

print('Standard error for a1 estimate = ', stderr_a1_mle)
print('Standard error for a2 estimate = ', stderr_a2_mle)
print('Standard error for a3 estimate = ', stderr_a3_mle)
print('Standard error for bf estimate = ', stderr_bf_mle)
print('Standard error for bp estimate = ', stderr_bp_mle)
print('Standard error for theta estimate = ', stderr_theta_mle)

Standard error for a1 estimate =  0.06671632055044982
Standard error for a2 estimate =  0.02959206917454104
Standard error for a3 estimate =  0.07896443344265563
Standard error for bf estimate =  0.029591624905351072
Standard error for bp estimate =  2.605273084327706
Standard error for theta estimate =  0.08175547577415325


#### 4. brands 1 and 3 in one nest; 2 and 4 in another

In [16]:
def nestedlogit4(params):

    a1,a2,a3,bf,bp,theta1,theta2=params
    rho1=np.exp(theta1)/(np.exp(theta1)+1)
    rho2=np.exp(theta2)/(np.exp(theta2)+1)
    
    #Nest1 (Brand 1 and 2)
    ev1=np.exp((a1+bf*df['Feature 1']+bp*df['Price 1'])/rho1)
    ev3=np.exp((a3+bf*df['Feature 3']+bp*df['Price 3'])/rho1)
    denom1=ev1+ev3

    #Nest2 (Brand 3 and 4)
    ev2=np.exp((a2+bf*df['Feature 2']+bp*df['Price 2'])/rho2)
    ev4=np.exp((0+bf*df['Feature 4']+bp*df['Price 4'])/rho2)
    denom2=ev2+ev4
      
    Nest1=np.power(denom1,rho1)/(np.power(denom1,rho1)+np.power(denom2,rho2))
    Nest2=np.power(denom2,rho2)/(np.power(denom1,rho1)+np.power(denom2,rho2))

    P1=Nest1*ev1/denom1
    P2=Nest2*ev2/denom2
    P3=Nest1*ev3/denom1
    P4=Nest2*ev4/denom2
     
    pc=(P1*df['Brand 1']+P2*df['Brand  2']+P3*df['Brand 3']+P4*df['Brand 4'])
    Inpc=np.log(pc)
    LL=np.sum(Inpc)
    return -LL

In [17]:
a1,a2,a3,bf,bp,theta1,theta2=1,1,1,1,1,1,1 #Initalization
params_init = np.array([a1,a2,a3,bf,bp,theta1,theta2])
results = opt.minimize(nestedlogit4, params_init)
a1,a2,a3,bf,bp,theta1,theta2 = results.x
print(" a1 (Intrinsic brand preference for Brand 1):",a1,"\n",
      "a2 (Intrinsic brand preference for Brand 2):",a2,"\n",
      "a3 (Intrinsic brand preference for Brand 3):",a3,"\n",
      "bf (Coefficients for feature variable):",bf,"\n",
      "bp (Coefficients for price variable):",bp,"\n",
      "rho (Correlation variable 1):",np.exp(theta1)/(np.exp(theta1)+1),"\n",
      "rho (Correlation variable 2):",np.exp(theta2)/(np.exp(theta2)+1),"\n",
      "Maximized Log Likelihood:",-results.fun)

 a1 (Intrinsic brand preference for Brand 1): 1.3812460673010283 
 a2 (Intrinsic brand preference for Brand 2): 0.6423131393782008 
 a3 (Intrinsic brand preference for Brand 3): -2.847026599694185 
 bf (Coefficients for feature variable): 0.49156346089164277 
 bp (Coefficients for price variable): -36.508609848585486 
 rho (Correlation variable 1): 0.901615703802694 
 rho (Correlation variable 2): 0.9999990433380485 
 Maximized Log Likelihood: -2658.3966631253315


In [18]:
vcv_mle = results.hess_inv
stderr_a1_mle = np.sqrt(vcv_mle[0,0])
stderr_a2_mle = np.sqrt(vcv_mle[1,1])
stderr_a3_mle = np.sqrt(vcv_mle[2,2])
stderr_bf_mle = np.sqrt(vcv_mle[3,3])
stderr_bp_mle = np.sqrt(vcv_mle[4,4])
stderr_theta_mle = np.sqrt(vcv_mle[5,5])

print('Standard error for a1 estimate = ', stderr_a1_mle)
print('Standard error for a2 estimate = ', stderr_a2_mle)
print('Standard error for a3 estimate = ', stderr_a3_mle)
print('Standard error for bf estimate = ', stderr_bf_mle)
print('Standard error for bp estimate = ', stderr_bp_mle)
print('Standard error for theta estimate = ', stderr_theta_mle)

Standard error for a1 estimate =  0.08316113798127142
Standard error for a2 estimate =  0.04824393427177462
Standard error for a3 estimate =  0.37193832604676147
Standard error for bf estimate =  0.11037880841183746
Standard error for bp estimate =  2.414625402671832
Standard error for theta estimate =  1.6174746186369395


### Summary

In [19]:
N1=["1,2,3","1,4","1,2","1,3"]
N2=["4","2,3","3,4","2,4"]
r1=[0.64,0.92,0.70,0.90]
r2=["/",0.56,0.53,0.99]
table=pd.DataFrame({"Nest 1":N1,"Nest 2":N2,"rho1":r1,"rho2":r2})
table.index+=1
table

,Nest 1,Nest 2,rho1,rho2
1,"1,2,3",4,0.64,/
2,"1,4","2,3",0.92,0.56
3,"1,2","3,4",0.70,0.53
4,"1,3","2,4",0.90,0.99


In model 4, rho 1 and rho 2 are close to 1 and the coefficients are identical to the previous coefficients from the simple logit model. This suggests the model satisfies the i.i.a assumption.

In model 2 and 3, rho2 are relatively small (away from 1). This suggests the nested models violate the i.i.a assumption.